# Week 1 Lab: next generation sequencing and mutation hunting
## Prelab 2
**Due: Thursday 04/04/19 10am**

Today's prelab will focus on understanding **sequencing coverage** and how to model coverage in the presence of **genetic variants** from the reference genome.

## 1. Exploring sequencing coverage

In next generation sequencing (NGS), coverage refers to the number of reads *covering* a particular base pair (bp) in the reference genome.

Recall from lecture that we sequencing a genome in many little chunks, called *reads*. In a typical NGS experiment, reads are around 100bp each. We can assume that the start position of each read is chosen randomly from the genome (although that's not quite true). We usually produce enough sequencing data such that we cover each base in the genome many times. This allows us to more accurately correct for errors and to detect **variants** from the reference genome.

As a toy example, consider the following reference genome and read alignment:

```
Ref:    AACTACGTACGTACGATCAGACTG
Read 1:      CGTACG
Read 2:          CGTACG
Read 3:                  CAGACT
```

The 11th position (G in the reference) is covered 3 times (often denoted as 3x). The first position (A in the reference) is covered 0 times. 

<font color="red" size="3" style="background-color:#f2e3e5">**Question 1 (1 pts):** How many times is the 13th base covered?</font>

<font color="blue" size="3" style="background-color:#e2e6f1">**Answer 1**</font>

We can easily compute the *average coverage*, or mean number of times each base is covered, given the read length ($l$), genome size ($G$), and number of reads ($N$) by dividing the number of total sequenced bases ($l*N$) by the total genome size ($G$): 

$$\texttt{Mean Coverage} = \frac{lN}{G}$$

<font color="red" size="3" style="background-color:#f2e3e5">**Question 2 (2 pts):** For a genome of total length 2000bp, you sequenced 100,000 reads with length 36. What is the mean coverage?</font>

<font color="blue" size="3" style="background-color:#e2e6f1">**Answer 2**</font>

<font color="red" size="3" style="background-color:#f2e3e5">**Question 3 (2 pts):** You would like to sequence a human genome (3 billion bp) to mean coverage 50x. Approximately how many 100bp reads should you generate?</font>

<font color="blue" size="3" style="background-color:#e2e6f1">**Answer 3**</font>

## 2. Reviewing the binomial distribution

TODO include p-value distribution